In [ ]:
import numpy as np
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split

# Define image size
IMG_SIZE = (224, 224)  # MobileNetV2 requires 224x224 input size

# Function to load and preprocess image
def load_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, IMG_SIZE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Use MobileNetV2 preprocessing
    return img_array

# Load and preprocess all images from a directory
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img_array = load_image(img_path)
            images.append(img_array)
            labels.append(label)
    return np.vstack(images), np.array(labels)

# Load positive samples (hand sign images)
positive_folder = 'path'
positive_images, positive_labels = load_images_from_folder(positive_folder, 1)

# Load negative samples (non-hand sign images)
negative_folder = 'path'
negative_images, negative_labels = load_images_from_folder(negative_folder, 0)

# Combine and balance data
images = np.vstack((positive_images, negative_images))
labels = np.concatenate((positive_labels, negative_labels))

# Shuffle and split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the MobileNetV2 model, excluding the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom top layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(datagen.flow(X_train, y_train, batch_size=32), validation_data=(X_val, y_val), epochs=100)


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import time
import threading
import tNavigator_python_API as tnav
from tNavigator_python_API import ProjectType

# Function to preprocess the frame
def preprocess_frame(frame):
    img = cv2.resize(frame, (224, 224))  # Resize to 224x224 for MobileNetV2
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Use MobileNetV2 preprocessing
    return img_array

# Function to predict hand sign
def predict_hand_sign(model, frame):
    img = preprocess_frame(frame)
    prediction = model.predict(img)
    print(f"Prediction: {prediction[0][0]}")  # Debug: print the prediction value
    return prediction[0][0] > 0.5  # Return True if the prediction is above 0.5

# Function to draw bounding box and text
def draw_box_and_text(frame, text, color):
    h, w, _ = frame.shape
    cv2.rectangle(frame, (int(w*0.1), int(h*0.1)), (int(w*0.9), int(h*0.9)), color, 2)
    cv2.putText(frame, text, (int(w*0.3), int(h*0.9) + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

# Function to execute the given script
def execute_script():
    conn = tnav.Connection(path_to_exe='path')
    snp_project = conn.open_project(path='path')
    snp_project.run_py_code(code="path')")
    snp_project.run_py_code(file="path")

# Function to run the script in a separate thread
def run_script_thread():
    script_thread = threading.Thread(target=execute_script)
    script_thread.start()

# Open the camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

start_time = None
done_duration = 1  # Duration in seconds to hold "Done" state

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break
    
    # Predict hand sign
    is_sign_detected = predict_hand_sign(model, frame)
    if is_sign_detected:
        draw_box_and_text(frame, "Model_Is_Running_Now_:)", (0, 255, 0))  # Green box
        if start_time is None:
            start_time = time.time()
        elif time.time() - start_time >= done_duration:
            run_script_thread()
            break
    else:
        draw_box_and_text(frame, "Waiting_For_Ur_Permission", (0, 0, 255))  # Red box
        start_time = None
    
    # Display the frame
    cv2.imshow('Hand Sign Detection', frame)
    
    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
